In [1]:
# 导入pandas
import pandas as pd
# 导入数据
train=pd.read_csv('./archive/train.csv')
test=pd.read_csv('./archive/test.csv')

In [2]:
def preprocessing(dfdata):
 
    dfresult= pd.DataFrame()
 
    #Pclass
    dfPclass = pd.get_dummies(dfdata['Pclass'])
    dfPclass.columns = ['Pclass_' +str(x) for x in dfPclass.columns ]
    dfresult = pd.concat([dfresult,dfPclass],axis = 1)
 
    #Sex
    dfSex = pd.get_dummies(dfdata['Sex'])
    dfresult = pd.concat([dfresult,dfSex],axis = 1)
 
    #Age
    dfresult['Age'] = dfdata['Age'].fillna(0)
    dfresult['Age_null'] = pd.isna(dfdata['Age']).astype('int32')
 
    #SibSp,Parch,Fare
    dfresult['SibSp'] = dfdata['SibSp']
    dfresult['Parch'] = dfdata['Parch']
    dfresult['Fare'] = dfdata['Fare']
 
    #Carbin
    dfresult['Cabin_null'] =  pd.isna(dfdata['Cabin']).astype('int32')
 
    #Embarked
    dfEmbarked = pd.get_dummies(dfdata['Embarked'],dummy_na=True)
    dfEmbarked.columns = ['Embarked_' + str(x) for x in dfEmbarked.columns]
    dfresult = pd.concat([dfresult,dfEmbarked],axis = 1)
    
    #Fare
    dfresult['Fare'].fillna(dfresult['Fare'].mean(),inplace=True)
    
    #归一化处理
    dfresult['Age']=dfresult['Age']/100
    dfresult['Fare']=dfresult['Fare']/100
    
    return(dfresult)

In [3]:
def pv(data):
    dfdata=preprocessing(data)
    result1=[]
    for i in range(len(dfdata)):
        result1.append(dfdata[i:i+1])
    result1=np.array(result1)
    result2=[]
    for i in range(len(result1)):
        result2.append(result1[i:i+1])
    dfresult=np.array(result2)
    return dfresult

In [4]:
import numpy as np
dfresult= pd.DataFrame()
Survived=pd.get_dummies(train['Survived'])
Survived.columns = ['Survived_' + str(i) for i in Survived.columns]
y = pd.concat([dfresult,Survived],axis = 1)
y = np.array(y)
print(y.shape)

(891, 2)


In [5]:
X=pv(train)
X_test=pv(test)
print(X.shape)

(891, 1, 1, 15)


In [6]:
from sklearn.model_selection import train_test_split # 导入拆分工具
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=0)

In [7]:
from keras import layers
from keras import models
from keras import optimizers
model=models.Sequential()
model.add(layers.Conv2D(32,(1,1),activation='relu',input_shape=(1,1,15)))
model.add(layers.MaxPool2D(2,2, 'same'))
model.add(layers.Conv2D(64,(1,1),activation='relu')) # 卷积
model.add(layers.MaxPool2D(2,2, 'same'))
model.add(layers.Conv2D(128,(1,1),activation='relu')) # 卷积
model.add(layers.MaxPool2D(2,2, 'same'))
model.add(layers.Conv2D(128,(1,1),activation='relu')) # 卷积
model.add(layers.MaxPool2D(2,2, 'same'))
model.add(layers.Flatten()) # 展平
model.add(layers.Dense(512,activation='relu')) # 全连层
model.add(layers.Dense(2,activation='softmax')) # 分类输出
model.compile(loss='categorical_crossentropy', # 损失函数
           optimizer='RMSprop', # 优化器
           metrics=['acc']) # 评估指标

In [8]:
history = model.fit(X_train,y_train, # 指定训练集
                 epochs=50,        # 指定轮次
                 batch_size=256,   # 指定批量大小
                 validation_data=(X_val,y_val)) # 指定验证集

Epoch 1/50
3/3 [==============================] - 1s 76ms/step - loss: 0.6528 - acc: 0.5604 - val_loss: 0.6263 - val_acc: 0.6927
Epoch 2/50
3/3 [==============================] - 0s 10ms/step - loss: 0.5724 - acc: 0.7430 - val_loss: 0.4784 - val_acc: 0.8101
Epoch 3/50
3/3 [==============================] - 0s 11ms/step - loss: 0.4932 - acc: 0.7893 - val_loss: 0.4697 - val_acc: 0.7709
Epoch 4/50
3/3 [==============================] - 0s 10ms/step - loss: 0.5035 - acc: 0.7809 - val_loss: 0.4521 - val_acc: 0.7989
Epoch 5/50
3/3 [==============================] - 0s 13ms/step - loss: 0.4703 - acc: 0.7992 - val_loss: 0.4238 - val_acc: 0.8101
Epoch 6/50
3/3 [==============================] - 0s 13ms/step - loss: 0.4501 - acc: 0.7963 - val_loss: 0.4186 - val_acc: 0.8045
Epoch 7/50
3/3 [==============================] - 0s 12ms/step - loss: 0.4705 - acc: 0.7921 - val_loss: 0.4275 - val_acc: 0.7989
Epoch 8/50
3/3 [==============================] - 0s 11ms/step - loss: 0.4321 - acc: 0.8034 - val

In [9]:
from sklearn.metrics import classification_report
evaluate_result=model.evaluate(X_val,y_val)
print("评估：",evaluate_result)

y_pred=model.predict(X_val)
for i in range(len(y_pred)):
    if y_pred[i][0]>0.5:
        y_pred[i][0]=1
        y_pred[i][1]=0
    else:
        y_pred[i][0]=0
        y_pred[i][1]=1
print("评估：")
print(classification_report(y_val,y_pred))

6/6 [==============================] - 0s 1ms/step - loss: 0.4662 - acc: 0.7821
评估： [0.4661563038825989, 0.7821229100227356]
评估：
              precision    recall  f1-score   support

           0       0.81      0.84      0.83       110
           1       0.73      0.70      0.71        69

   micro avg       0.78      0.78      0.78       179
   macro avg       0.77      0.77      0.77       179
weighted avg       0.78      0.78      0.78       179
 samples avg       0.78      0.78      0.78       179



In [10]:
y_test_pred=model.predict(X_test)
pred=[]
for i in range(len(y_test_pred)):
    if y_test_pred[i][0]>0.5:
        y_test_pred[i][0]=1
        y_test_pred[i][1]=0
        pred.append(y_test_pred[i][1])
    else:
        y_test_pred[i][0]=0
        y_test_pred[i][1]=1
        pred.append(y_test_pred[i][0])
print(y_test_pred[0:10])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [11]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [12]:
pred_result=pd.DataFrame()
pred_result['Survived']=pred
pred_Survived=pd.concat([test,pred_result],axis=1)

In [13]:
# 对test数据集的预测结果
pred_Survived

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0.0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,0.0
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0.0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0.0
